In [1]:
import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt
%matplotlib inline

## Arithmetical operations

So far all the operations on tensors did not change their elements. Time to change this. 

`numpy` supports all arithmetic operations and many functions in form of elementwise operations. For example  for multiplication

$$ \forall_{ijk}\quad  z_{ijk}=x_{ijkl} \cdot y_{ijk}$$ 

Please note several different ways used to time the commands below. 

In [38]:
x = np.random.normal(0,1,(3,5,2))
y = np.random.normal(0,1,(3,5,2))
z = x * y 

Please note that arithmetic operations create a new array. 

In [39]:
print(z.base)

None


This is equivalent to the following loop but faster

In [4]:
%%time
zloop = np.zeros((3,5,2))
for i in range(3):
    for j in range(5):
        for k in range(2):
            z[i,j,k]=x[i,j,k] * y[i,j,k]

Wall time: 1 ms


Time difference in this case is not  very big, but for bigger arrays it can becomes very large:

In [5]:
import timeit

In [6]:
xb = np.random.normal(0,1,(30,50,20))
yb = np.random.normal(0,1,xb.shape)
start_time = timeit.default_timer()
zb = xb * yb 
end_time = timeit.default_timer()
elapsed_implicit = end_time-start_time
print("Took %s " % (elapsed_implicit,))

Took 0.000266855999999871 


In [7]:
s = xb.shape
start_time = timeit.default_timer()
zbloop = np.empty_like(xb)
for i in range(s[0]):
    for j in range(s[1]):
        for k in range(s[2]):
            zbloop[i,j,k]=xb[i,j,k] * yb[i,j,k]
end_time = timeit.default_timer()            
elapsed_explicit = end_time-start_time
print("Took %fs which is %f times longer!" %(elapsed_explicit, elapsed_explicit/elapsed_implicit))

Took 0.021021s which is 78.774313 times longer!


As you can see the difference is of the order of  hundreds!  That is the main reason you shoudl become fluent in tensor operations.

Similarly we can apply a numpy function to every element of the tensor just by calling it with tensor argument:

$$\forall_{ijk}\quad s_{ijk} = \sin(x_{ijk})$$ 

In [8]:
%time s = np.sin(x)

Wall time: 0 ns


Please compare yourself the time of the execution of this operation to an explicit loop. 

You can also use a scalar argument in tensor operations with the common sense interpretation:

In [9]:
grumpy = img.imread("GrumpyCat.jpg")

In [10]:
normalized_grumpy = grumpy/255

## Reduction

Another common operations are  reductions. Those are the functions that can be applied to a subset of dimensions "reducing" them  to a single number. Using our freshly acquired skills in array manipulations we will build an array where every column will contain 1000 numbers draw from a different distribution. 

In [11]:
n = 1000
d1 = np.random.normal(0,1, n)
d2 = np.random.normal(1,0.5, n)
d3 =np.random.uniform(0,1,n)
data = np.stack((d1,d2,d3), axis=1)
data.shape

(1000, 3)

A common reduction operation is sum. Without any additional parameters sum sums all the element of the array

In [12]:
np.sum(data)

1508.4070224525453

But we can specify the dimension(s) along which the reduction operation will be applied. 

In [13]:
row_sum = np.sum(data, axis=1)
row_sum.shape

(1000,)

As we can see the dimension 1 was "reduced". 

In the same way we can calculate the mean of every column:

In [14]:
np.mean(data, axis=0)

array([0.00760306, 0.99573751, 0.50506646])

or standard deviation

In [15]:
np.std(data, axis=0)

array([1.00578399, 0.49083516, 0.28899063])

We can reduce more then one dimension at the time. Below we calculate the mean value of each chanel in grumpy

In [16]:
np.mean(grumpy, axis=(0,1))

array([117.03265727, 103.90124729, 105.09348518])

or max and min  values

In [17]:
np.min(grumpy, axis=(0,1))

array([0, 0, 0], dtype=uint8)

In [18]:
np.max(grumpy, axis=(0,1))

array([255, 255, 255], dtype=uint8)

## Contractions -- inner product

Another class of operations are contraction. In contraction we sum over two dimensions of a product of two arrays. The examples include the dot (scalar) product

$$ x\cdot y =\sum_{i} x_{i} \cdot y_{i}$$ 

matrix vector multiplication:

$$ v_j =\sum_{i} A_{ji} \cdot w_{i} \quad \forall_{i}$$ 

and matrix multiplication

$$   z_{ij}=\sum_{k} x_{ik} \cdot y_{kj} \quad \forall_{ij}$$ 

`numpy` has special operators for both operations but we can use more general `inner` and `tensordot`. 

`inner` takes two arrays and contracts last dimensions in each of them. That means that the sizes of those dimensions must match. 

When both arrays are vectors this is normal scalar product:

In [19]:
x = np.random.normal(0,1,10)
y = np.ones_like(x)
np.inner(x,y)

-0.3445058287709808

When first is  a matrix and other a vector this is matrix vector multiplication:

In [20]:
m = np.asarray([[1,-1],[-1,1]])
v = np.array([0.5, -0.5])
np.inner(m,v)

array([ 1., -1.])

Can you tell what the operation below is doing? 

In [40]:
w =np.asarray([0.3, 0.59, 0.11])
G = np.inner(grumpy,w)
print(G)

[[  0.     0.     0.   ... 117.84 110.83 112.04]
 [  0.     0.     0.   ... 115.84 108.83 111.04]
 [  0.     0.     0.   ... 115.84 107.83 110.34]
 ...
 [  0.     0.     0.   ... 185.05 186.05 186.05]
 [  0.     0.     0.   ... 180.05 181.05 183.05]
 [  0.     0.     0.   ... 179.05 179.05 179.76]]


Similar to `inner` is `dot`.Please check out its documentatio [here](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html). 

Matrix multiplication requires contraction of last and first dimension. That's why it's more convenient to use `tensordot`

In [22]:
A = np.random.normal(0,1,(2,3))
B = np.random.normal(0,2,(3,4))
C = np.tensordot(A,B,1)

In [23]:
print(C.shape)
C

(2, 4)


array([[-7.46083223, -0.06284435,  2.3447464 , -0.85201796],
       [ 2.27517279,  0.22029371, -0.65500151,  1.58621055]])

If we want to do matrix multiplication it's better to use 
`matmul` function which is described [here](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html#numpy.matmul). This function can be invokde using operator `@`

In [24]:
A@B

array([[-7.46083223, -0.06284435,  2.3447464 , -0.85201796],
       [ 2.27517279,  0.22029371, -0.65500151,  1.58621055]])

`tensordot(A,B,n)` is more general contracts last `n` dimensions of array `A` with first `n` dimensions of array `B`. 

In [25]:
A2 = np.random.normal(0,1,(4,3))
B2 = np.random.normal(0,2,(4,3))
C2 = np.tensordot(A2,B2,2)
print(C2.shape)
C2

()


array(-1.09103697)

In the above expression `C2` is calculated as: 
$$ C = \sum_{ij}A_{ij} B_{ij}$$

We can also specify which dimensions will be contracted, by providing lists of dimensions in each array:

In [26]:
A3 = np.random.normal(0,1,(4,3))
B3 = np.random.normal(0,2,(3,4))
C3 = np.tensordot(A3,B3,[[0,1], [1,0]])
print(C3.shape)
C3

()


array(3.0107933)

Which corresponds to 
$$ C = \sum_{ij}A_{ij} B_{ji}$$
which is more intuitive.

You have a matrix 3x4 matrix W and a set of N 4-vectors in a form of array X of shape (N,4). How to produce an array of shape (N,3) where each row is the product of matrix W and corresponding row of X ? 

## Outer product

What happens when we request zero dimension contraction in `tensordot`? For two vectors this should correspond to
$$ z_{ij} = x_i \cdot y_j\quad \forall_{ij} $$
Let's check this. 

In [27]:
x = np.arange(4)
y = np.arange(5)
z  = np.tensordot(x,y,0)
print(z.shape)
z

(4, 5)


array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12]])

This operation is called outer or tensor product. We can achieve same result with function `outer`

In [28]:
x = np.arange(4)
y = np.arange(5)
z  = np.outer(x,y)
print(z.shape)
z

(4, 5)


array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12]])

However those two functions behave the same same only for 1-dimensional arrays. 

## "Degenerate" dimensions

This a technical but a quite important point. It concerns dimensions with size one. While it may seem that such dimensions are spurious or "degenerate" they nevertheless change the dimensionality of the array and can impact the result of the operations.

Let's start by creating a vector

In [29]:
vector = np.random.normal(0,1,(4,))
print(vector.shape)
vector

(4,)


array([-0.03326301, -0.71418614, -1.2868607 , -0.37267573])

and reshape it to one row matrix 

In [30]:
vector_row = np.reshape(vector,(1,4))
print(vector_row.shape)
vector_row

(1, 4)


array([[-0.03326301, -0.71418614, -1.2868607 , -0.37267573]])

and one column matrix:

In [31]:
vector_column = np.reshape(vector,(4,1))
print(vector_column.shape)
vector_column

(4, 1)


array([[-0.03326301],
       [-0.71418614],
       [-1.2868607 ],
       [-0.37267573]])

Now make some experiments:

In [32]:
np.inner(vector, vector)

2.306065926125458

In [33]:
np.inner(vector_row, vector_row)

array([[2.30606593]])

In [34]:
np.inner(vector_column, vector_column)

array([[1.10642792e-03, 2.37559816e-02, 4.28048620e-02, 1.23963169e-02],
       [2.37559816e-02, 5.10061840e-01, 9.19058074e-01, 2.66159837e-01],
       [4.28048620e-02, 9.19058074e-01, 1.65601046e+00, 4.79581745e-01],
       [1.23963169e-02, 2.66159837e-01, 4.79581745e-01, 1.38887197e-01]])

This actually the outer product:

In [35]:
np.outer(vector, vector)

array([[1.10642792e-03, 2.37559816e-02, 4.28048620e-02, 1.23963169e-02],
       [2.37559816e-02, 5.10061840e-01, 9.19058074e-01, 2.66159837e-01],
       [4.28048620e-02, 9.19058074e-01, 1.65601046e+00, 4.79581745e-01],
       [1.23963169e-02, 2.66159837e-01, 4.79581745e-01, 1.38887197e-01]])

The only two other combinations that will match are: 

In [36]:
np.inner(vector, vector_row)

array([2.30606593])

In [37]:
np.inner(vector_row, vector)

array([2.30606593])

Please explain the results of all the above operations. Write down using indices what each operation actually does. 